In [ ]:
import matplotlib.pyplot as plt
from skimage import data, color, feature, io
from skimage.transform import resize 
import numpy as np

In [ ]:

def hog_feature(image, pixel_per_cell=8):
    """
    Compute HOG feature for a given image.

    Args:
        image: an image with object that we want to detect.
        pixel_per_cell: number of pixels in each cell, an argument for hog descriptor.

    Returns:
        hogFeature: a vector of hog representation.
        hogImage: an image representation of hog provided by skimage.
    """
    hogFeature, hogImage = feature.hog(image,
                                       pixels_per_cell=(pixel_per_cell, pixel_per_cell),
                                       cells_per_block=(1, 1),
                                       block_norm='L1',
                                       visualize=True,
                                       feature_vector=True)
    return hogFeature, hogImage

In [ ]:
# Load an example image (you can replace this with your own image)
image = io.imread('test2.png')  # Thay 'your_image_path.png' bằng đường dẫn đến ảnh của bạn
image = resize(image, (image.shape[0] // 6, image.shape[1] // 6))
try :
    image = color.rgb2gray(image)
except:
    pass

# Compute HOG features and HOG image
hogFeature, hogImage = hog_feature(image)

# Display results
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Input image')
ax[0].axis('off')

ax[1].imshow(hogImage, cmap=plt.cm.gray)
ax[1].set_title('HOG image')
ax[1].axis('off')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from skimage import feature, io
from skimage.transform import resize
import numpy as np

def hog_feature(image, pixel_per_cell=8):
    """
    Compute HOG feature for a given image.

    Args:
        image: an image with object that we want to detect.
        pixel_per_cell: number of pixels in each cell, an argument for hog descriptor.

    Returns:
        hogFeature: a vector of hog representation.
        hogImage: an image representation of hog provided by skimage.
    """
    hogFeature, hogImage = feature.hog(image,
                                       pixels_per_cell=(pixel_per_cell, pixel_per_cell),
                                       cells_per_block=(3, 3),
                                       block_norm='L1',
                                       visualize=True,
                                       feature_vector=True)
    return hogFeature, hogImage

def sliding_window(image, base_score, stepSize, windowSize, pixel_per_cell=8):
    """ A sliding window that checks each different location in the image,
        and finds which location has the highest hog score. The hog score is computed
        as the dot product between the hog feature of the sliding window and the hog feature
        of the template.

    Args:
        image: an np array of size (h,w).
        base_score: hog representation of the object you want to find, an array of size (m,).
        stepSize: an int of the step size to move the window.
        windowSize: a pair of ints that is the height and width of the window.

    Returns:
        max_score: float of the highest hog score.
        maxr: int of row where the max_score is found (top-left of window).
        maxc: int of column where the max_score is found (top-left of window).
        response_map: an np array of size (h,w).
    """
    (max_score, maxr, maxc) = (0, 0, 0)
    winH, winW = windowSize
    H, W = image.shape
    response_map = np.zeros((H // stepSize + 1, W // stepSize + 1))

    for r in range(0, H - winH + 1, stepSize):
        for c in range(0, W - winW + 1, stepSize):
            window = image[r:r + winH, c:c + winW]
            hogFeature, _ = hog_feature(window, pixel_per_cell)
            score = np.dot(hogFeature, base_score)
            response_map[r // stepSize, c // stepSize] = score
            if score > max_score:
                max_score = score
                maxr, maxc = r, c

    response_map = resize(response_map, (H, W))

    return (max_score, maxr, maxc, response_map)

# Load an example grayscale image
image = io.imread('imagedata/00/01_palm/frame_00_01_0001.png', as_gray=True)

# Define the template (for example, a hand region) - here we assume you have a predefined template
# You need to load or define your own template for hand detection
template = io.imread('path_to_template_image.png', as_gray=True)
template_hog, _ = hog_feature(template)

# Parameters
stepSize = 20
windowSize = template.shape

# Perform sliding window to detect the region
max_score, maxr, maxc, response_map = sliding_window(image, template_hog, stepSize, windowSize)

# Draw a rectangle around the detected region
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.imshow(image, cmap=plt.cm.gray)
rect = plt.Rectangle((maxc, maxr), windowSize[1], windowSize[0], edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()

# Display the response map
plt.imshow(response_map, cmap='hot')
plt.title('Response Map')
plt.show()


In [59]:
import cv2
import numpy as np

# Đọc ảnh vào
image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)

# Áp dụng ngưỡng hóa
_, binary_image = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)

# Hiển thị ảnh kết quả
cv2.imshow('Binary Image', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
import cv2

# Đọc ảnh vào
image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)

# Áp dụng ngưỡng hóa thích ứng
adaptive_thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# Hiển thị ảnh kết quả
cv2.imshow('Adaptive Threshold Image', adaptive_thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [52]:
import cv2

# Đọc ảnh vào
image = cv2.imread('test2.png', cv2.IMREAD_GRAYSCALE)

# Áp dụng cân bằng lược đồ màu
equalized_image = cv2.equalizeHist(image)

# Hiển thị ảnh kết quả
cv2.imshow('Equalized Image', equalized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [39]:
import cv2
import numpy as np

# Đọc ảnh vào
image = cv2.imread('test2.png')

# Chuyển đổi ảnh sang float32
image = np.float32(image) / 255.0

# Điều chỉnh gamma
gamma = 0.5
corrected_image = np.power(image, gamma)

# Chuyển đổi lại về uint8
corrected_image = np.uint8(corrected_image * 255)

# Hiển thị ảnh kết quả
cv2.imshow('Gamma Corrected Image', corrected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [65]:
import cv2
import numpy as np

# Đọc video từ camera
cap = cv2.VideoCapture(0)

# Khởi tạo kích thước cửa sổ

# Khởi tạo bộ phân đoạn nền
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    # Đọc từng khung hình
    ret, frame = cap.read()
    if not ret:
        break
    
    # Áp dụng bộ phân đoạn nền để phát hiện vật thể đang chuyển động
    fgmask = fgbg.apply(frame)
    
    # Xử lý để loại bỏ nhiễu và cải thiện kết quả
    kernel = np.ones((5,5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    # Tìm các đường viền của các vật thể đã phát hiện
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Vẽ đường viền cho vật thể lớn nhất
    if len(contours) > 0:
        max_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(max_contour)
        cv2.rectangle(frame, (x, y), (x+w, y+w), (0, 255, 0), 2)
    
    # Hiển thị video kết quả
    cv2.imshow('Motion Detection', frame)


    
    # Thoát nếu nhấn phím 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên và đóng cửa sổ
cap.release()
cv2.destroyAllWindows()
